* Python与World   
[20210619 created, 20210620 last updated]    

Word和Excel一样,微软公司为Windows系统开发的一款软件.Excel用于处理电子表格,而Word则是一款优秀的文字编辑工具,和它类似的软件有Mac OS上的Pages,国产的WPS和开源的open office,Librel office等等,他们都可以用于处理后缀为docx的文件.    
不同于文本文档txt,word所处理的后缀为.doxc的文件也是一种二进制文件,除了文本之外还保存了许多字体,颜色以及布局信息,它本质上也是可以看作是一个xml文档的,这也是很多非word软件能够制作出被word兼容的后缀为docx的文件见的原因.   
例如,我们把如下的文本内容复制粘贴到名为txt_doc.txt文本文档内,然后把后缀名改为.docx,那么我们就可以使用word等工具打开这个名为txt_doc.docx文件了.    

`
<?xml version="1.0"?>
<w:wordDocument xmlns:w="http://schemas.microsoft.com/office/word/2003/wordml">
<w:body>
<w:p>
<w:r>
<w:t>Hello, World.</w:t>
</w:r>
</w:p>
</w:body>
</w:wordDocument>
`

![使用xml制作的docx文档](./图片/使用xml格式制作的docx文档.png)

基于这样的认识,我们就可以"以读取和修改xml格式文件中的标签内的内容"的方式来读写Word文档了.当然,具体的解析操作我们交给专门的Python第三方库去完成,我们把注意力集中于使用这个库中的方法对word文件进行一些基本的操作.    

Python 可以利用 python-docx 模块创建和修改带有.docx后缀名的 Word 文档. 运行 pip install python-docx 以安装该模块(在使用 pip 安装 python-docx 时安装的模块名称是 python-docx，而不是 docx,安装名称 docx 是指另一个模块). 但是在导入 python-docx 模块时,需要执行 import docx,而不是 import python-docx.

In [1]:
#安装docx库
!pip3 install python-docx


  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184606 sha256=2db04fcbd70bd0861ea848dba44dcb905bd5bcd4682b6d49fdd68835d6b93f29
  Stored in directory: c:\users\liby\appdata\local\pip\cache\wheels\32\b8\b2\c4c2b95765e615fe139b0b17b5ea7c0e1b6519b0a9ec8fb34d
Successfully built python-docx


# 读取Word文档

我们在上边提到,和纯文本相比,.docx文件有很多结构, 它的页面结构可以分为三层:     
* 第一层: **文档(Document)**;       
    python-docx将整个文章看做是一个Document对象,一个Word文件就是一个Documment类型的对象;
* 第二层: **段落(Paragraph)**; 
    每个Document包含许多个代表“段落”的Paragraph对象,存放在document.paragraphs中;
* 第三层: **文字块(Run)**.     
    每个Paragraph都有许多个代表"行内元素"的Run对象,存放在paragraph.runs中.    
    在python-docx中，run是最基本的单位，每个run对象内的文本样式都是一致的，也就是说，在从docx文件生成文档对象时，python-docx会根据样式的变化(包括字体,大小,颜色和其他样式等)来将文本切分为一个个的Run对象,一个Run对象是相同样式文本组成的字符串(当然也包含了样式).当文本样式发生变换后,就形成了一个新的Run对象,不论上一个Run对象包含了几个字符.



#### 打开已经存在的Word文档
在Python中,使用docx模块的Document方法,可以打开指定路径处的Word文档.    
和Excel时的情况类似,用Document方法打开Word后,实际上是创建了一个Document类型的对象,如果需要获取Word文档的内容,还需要进一步使用该对象的其他方法.

In [75]:
doc_0 = Document(r'D:\Py\OfficeAutomation\data1\word\doc_0.docx')

In [76]:
type(doc_0)

docx.document.Document

In [77]:
# 查看Document对象的属性和方法
print(dir(doc_0))

['_Document__body', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_block_width', '_body', '_element', '_parent', '_part', 'add_heading', 'add_page_break', 'add_paragraph', 'add_picture', 'add_section', 'add_table', 'core_properties', 'element', 'inline_shapes', 'paragraphs', 'part', 'save', 'sections', 'settings', 'styles', 'tables']


#### 获取docx文档包含的段落
通过Document类型对象的Pragraphs属性,可以获得它所包含的段落构成的列表(list).文档包含了几个段落,list就有几个元素.   


In [78]:
doc_0.paragraphs

In [79]:
doc_0.paragraphs[0]

可以通过列表索引的方式来访问每一个段落.  
注意这时我们仍然不能看到文档包含的具体的文字内容.要获取文本,需要使用段落的属性.

In [80]:
par0 = doc_0.paragraphs[0]

In [73]:
print(dir(par0))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_element', '_insert_paragraph_before', '_p', '_parent', 'add_run', 'alignment', 'clear', 'insert_paragraph_before', 'paragraph_format', 'part', 'runs', 'style', 'text']


#### 获取Word文档中的文本
一个Word文档,包含了格式,文本,图片,表格等等各种类型的内容,但多数情况下我们最关注的是文档中的文本内容.   
在创建了Document类型的对象后,对于它所包含的每一个段落对象Paragraph,我们可以使用段落对象的text属性获取该段落包含的文本内容.

In [81]:
par0.text

'周杰伦'

如果要获取所有段落的文本,只需要遍历文档的所有段落,通过text属性获取每个段落的文本.

In [82]:
for par in doc_0.paragraphs:
    print(par.text,'\n')

周杰伦 

中国台湾华语流行歌手、音乐创作家、作曲家、作词人、制作人、杰威尔音乐公司老板之一、导演。近年涉足电影行业。 

周杰伦是2000年后亚洲流行乐坛最具革命性与指标性的创作歌手，有“亚洲流行天王”之称。他突破原有亚洲音乐的主题、形式，融合多元的音乐素材，创造出多变的歌曲风格，尤以融合中西式曲风的嘻哈或节奏蓝调最为著名，可说是开创华语流行音乐“中国风”的先声。 

周杰伦的出现打破了亚洲流行乐坛长年停滞不前的局面，为亚洲流行乐坛翻开了新的一页！ 



由于一篇docx文档的最小组成单位是Run对象,我们当然也可以从Run对象获取文档的所有文本,但显然没有从Paragraph对象获取文本方便.  
段落对象所包含的Run对象可以通过段落对象的runs属性获取,类似的,也是一个由Run对象构成的list.

In [83]:
par0.runs

In [84]:
print(dir(par0.runs))

['__add__', '__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [85]:
#获取Run对象的文本
par0.runs[0].text

'周杰伦'

In [87]:
# 从Run对象获取文本
for par in doc_0.paragraphs:
    for run in par.runs:
        print(run.text)

周杰伦
中国台湾华语流行歌手、音乐创作家、作曲家、作词人、制作人、杰威尔音乐公司老板之一、导演。
近年涉足电影行业。
周杰伦是2000年后亚洲流行乐坛最具革命性与指标性的创作歌手，有“亚洲流行天王”之称。他突破原有亚洲音乐的主题、形式，融合多元的音乐素材，创造出多变的歌曲风格，尤以融合中西式曲风的嘻哈或节奏蓝调最为著名，可说是开创华语流行音乐“中国风”的先声。
周杰伦的出现打破了亚洲流行乐坛长年停滞不前的局面，为亚洲流行乐坛翻开了新的一页！


In [ ]:
#### 获取文档中的表格/图片/

# 制作Word文件
使用Python制作Word文件,使得我们可以自动化生成或批量生成类似的Word文档,

## Document层面的制作

#### 新建空白word
和Excel的情形类似,使用Python新建一个word文档,就是创建一个docx的Document对象,这一步可以通过直接创建一个Document对象来实现

In [41]:
# 导入库
from docx import Document

In [42]:
# 新建空白文档
doc_1 = Document()

In [43]:
#又是一种新的数据类型对象
type(doc_1)

docx.document.Document

#### 保存word文件到磁盘
和使用鼠标右键新建word文档的不同之处在于,使用docx库新建Document对象之后,它是保存在内存中的,我们在当前工作目录是看不到的.想要将这个word文件保存到磁盘,还需要使用Document对象的`save`方法把这个Document对象保存到磁盘上的指定路径,并指定要保存的文件名,我们才能在当前工作目录中看到它.

In [44]:
# 保存文件（当前目录下）
doc_1.save('doc_1.docx')
#doc_1.save(r'D:\Py\OfficeAutomation\doc_1.docx') #也可以以相对路径或绝对路径的方式传入不同于当前工作目录的其他文件夹

这时候我们再查看当前工作目录,就会看到存在这么一个"doc_1.docx"的文件了.   
在查看完毕后,请记得关闭该文档,以便于我们继续使用Python来进行操作和保存这些操作到这个文档.

In [ ]:
import os

In [45]:
#使用os.path.exists方法检查文件保存结果
os.path.exists('doc_1.docx')

True

#### 添加标题


In [46]:
# 添加标题（0相当于文章的题目，默认级别是1，级别范围为0-9）
doc_1.add_heading('新建空白文档标题，级别为0',level = 0)
doc_1.add_heading('新建空白文档标题，级别为1',level = 1)
doc_1.add_heading('新建空白文档标题，级别为2',level = 2)

和新建文档需要保存才能在文件夹种看到它一样,要想使用 WPS 或 word 查看添加标题的结果,还是需要先把我们对Document对象的操作保存到磁盘.   
查看完毕记得关闭文档,以便于继续使用Python对文档进行修改.

In [47]:
doc_1.save('doc_1.docx')
#保存,然后使用word等工具查看修改结果

#### 添加图片
Document 对象的 add_picture()方法可以在文档末尾添加图像.但不像段落可以插入到指定位置,图片只能添加在文档结尾.

In [89]:
doc_0.add_picture??

Signature: doc_0.add_picture(image_path_or_stream, width=None, height=None)
Source:   
    def add_picture(self, image_path_or_stream, width=None, height=None):
        """
        Return a new picture shape added in its own paragraph at the end of
        the document. The picture contains the image at
        *image_path_or_stream*, scaled based on *width* and *height*. If
        neither width nor height is specified, the picture appears at its
        native size. If only one is specified, it is used to compute
        a scaling factor that is then applied to the unspecified dimension,
        preserving the aspect ratio of the image. The native size of the
        picture is calculated using the dots-per-inch (dpi) value specified
        in the image file, defaulting to 72 dpi if no value is specified, as
        is often the case.
        """
        run = self.add_paragraph().add_run()
        return run.add_picture(image_path_or_stream, width, height)
File:      d:\programdata

In [91]:
doc_0.add_picture('.\图片\参会人名单.png')

#### 添加表格*

## Paragraph层面的制作

#### 新增段落
为一个空白docx文档新增第一个段落,使用add_paragraph方法,并将文本通过字符串参数的方式传入就可以了.

In [48]:
paragraph_1 = doc_1.add_paragraph('这是第一段文字的开始\n请多多关照！')
# 在这里,使用了\n来进行换行.

In [49]:
doc_1.save('doc_1.docx') #这里可以自由设置,以自己所期望的文件名进行保存
#保存完毕后,就可以使用word等工具查看新增段落的结果了.

In [50]:
# 再试着使用长字符串增加一段
text = """
       中国台湾华语流行歌手、 
       音乐创作家、作曲家、作词人、 
       制作人、杰威尔音乐公司老板之一、导演。 
       近年涉足电影行业。周杰伦是2000年后亚洲流行乐坛最具革命性与指标 
       性的创作歌手，有“亚洲流行天王”之称。他突破原有亚洲音乐的主题、形 
       式，融合多元的音乐素材，创造出多变的歌曲风格，尤以融合中西式曲风的嘻哈 
       或节奏蓝调最为著名，可说是开创华语流行音乐“中国风”的先声。周杰伦的 
       出现打破了亚洲流行乐坛长年停滞不前的局面，为亚洲流行乐坛翻开了新的一页！

"""
#注意,这里我们并没有使用 \n 进行换行
paragraph_11 = doc_1.add_paragraph(text)

In [51]:
doc_1.save('doc_1.docx')
#保存后查看,发现word文件中的第二个段落实际上是由换行的,并且是按照我们在长字符串 text 中的设置那样进行换行的.

In [54]:
# 新增段落（会自动在当前段落的下方另起一段）
paragraph_2 = doc_1.add_paragraph('新起的第二段文字。')

#### 插入段落
对于 python-docx 的 0.5.3 版本,新的 Paragraph 对象只能添加在文档的末尾,新的 Run 对象只能添加在 Paragraph 对象的末尾. 但在较新的版本,可以将段落插入到指定段落之前. 

In [55]:
# 新段落插入到指定段落(paragraph_1)的上边
prior_paragraph = paragraph_1.insert_paragraph_before('在第一段文字前插入的段落')

#### 调整字体


In [52]:
# 加粗
paragraph_1.add_run('只需指定add_run方法的bold=True就可以得到加粗字体').bold = True
paragraph_1.add_run('不指定则得到普通字体')

In [53]:
# 斜体
paragraph_1.add_run('斜体字体').italic =True

In [56]:
doc_1.save('doc_1.docx')
#保存后进行查看

#### 添加换行符和分页符
要添加换行符(而不是开始一个新的段落)，可以在 Run 对象上调用 add_break()方法，换行符将出现在它后面。

如果希望添加分页符，可以将docx.text.WD_BREAK.PAGE作为唯一的参数，传递给add_break()，就像下面代码中间所做的一样：

In [ ]:
doc = docx.Document()
>>> doc.add_paragraph('This is on the first page!')
<docx.text.Paragraph object at 0x0000000003785518>
>>> doc.paragraphs[0].runs[0].add_break(docx.text.WD_ _BREAK.PAGE)
>>> doc.add_paragraph('This is on the second page!')
<docx.text.Paragraph object at 0x00000000037855F8>
>>> doc.save('twoPage.docx')

这创建了一个两页的 Word 文档，第一页上是 This is on the first page!，第二页
上是 This is on the second page!。虽然在文本 This is on the first page!之后，第一页还
有大量的空间，但是我们在第一段的第一个 Run 之后插入分页符，强制下一段落出
现在新的页面中.    


添加分页符,还可以使用add_page_break()

In [10]:
doc_1.add_page_break()

In [11]:
# 新段落（指定端的上方）
paragraph_3 = doc_1.add_paragraph('这是第二页第一段文字！')

In [12]:
# 如果需要使用word查看添加结果,需要先保存文件到硬盘
doc_1.save('doc_1.docx')

## Run层面的编辑 

# 项目实战

In [ ]:
# 导入库
from docx import Document
from docx.shared import RGBColor, Pt,Inches,Cm
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml.ns import qn

# 新建文档（Datawhale）
doc_1 = Document()

# 字体设置（全局）
'''只更改font.name是不够的，还需要调用._element.rPr.rFonts的set()方法。'''

doc_1.styles['Normal'].font.name = u'宋体'
doc_1.styles['Normal']._element.rPr.rFonts.set(qn('w:eastAsia'), u'宋体')

# 添加标题（0相当于文章的题目，默认级别是1，级别范围为0-9，0时候自动带下划线）
heading_1 = doc_1.add_heading('周杰伦',level = 0)
heading_1.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER   #居中对齐，默认居左对齐

# 新增段落
paragraph_1 = doc_1.add_paragraph()
'''
设置段落格式：首行缩进0.75cm，居左，段后距离1.0英寸,1.5倍行距。
'''
paragraph_1.paragraph_format.first_line_indent = Cm(0.75)
paragraph_1.paragraph_format.alignment =  WD_PARAGRAPH_ALIGNMENT.LEFT
paragraph_1.paragraph_format.space_after =  Inches(1.0)
paragraph_1.paragraph_format.line_spacing =  1.5

text = '中国台湾华语流行歌手、' \
       '音乐创作家、作曲家、作词人、' \
       '制作人、杰威尔音乐公司老板之一、导演。' \
       '近年涉足电影行业。周杰伦是2000年后亚洲流行乐坛最具革命性与指标' \
       '性的创作歌手，有“亚洲流行天王”之称。他突破原有亚洲音乐的主题、形' \
       '式，融合多元的音乐素材，创造出多变的歌曲风格，尤以融合中西式曲风的嘻哈' \
       '或节奏蓝调最为著名，可说是开创华语流行音乐“中国风”的先声。周杰伦的' \
       '出现打破了亚洲流行乐坛长年停滞不前的局面，为亚洲流行乐坛翻开了新的一页！'

r_1 = paragraph_1.add_run(text)
r_1.font.size =Pt(10)    #字号
r_1.font.bold =True       #加粗
r_1.font.color.rgb =RGBColor(255,0,0)      #颜色

# 保存文件（当前目录下）
doc_1.save('周杰伦.docx')

## 批量生成邀请函
根据excel中给定的名单,以及固定的邀请函的文本和格式